In [1]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [3]:
#pip install cursor

In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
#import cursor

In [2]:
order_df = pd.read_csv("data/orders.csv").sample(10000)

In [3]:
order_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
970247,3049511,58282,prior,11,3,11,6.0
1255483,3323833,75505,prior,2,0,10,28.0
164875,2300974,9950,prior,64,5,18,0.0
2279876,2920092,137311,prior,52,5,14,5.0
2320259,3349369,139723,prior,2,0,17,30.0


In [4]:
aisles_df = pd.read_csv("data/aisles.csv")
departments_df = pd.read_csv("data/departments.csv")
order_products_df = pd.read_csv("data/order_products.csv").sample(10000)
products_df = pd.read_csv("data/products.csv")

In [5]:
aisles_df.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [6]:
departments_df.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [7]:
order_products_df.head()

,order_id,product_id,add_to_cart_order,reordered
8539894,901532,35199,6,0
8829332,932046,9421,3,0
8462871,893325,35898,10,0
9586648,1011921,21903,11,1
2614940,275761,19820,14,0


In [8]:
products_df.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [18]:
#connection

try:
    conn = psycopg2.connect(host="localhost", dbname="", user="", password="", port=5432)
except:
    print("connection was unsuccessful")
#engine = create_engine('postgresql://:@localhost:5432/')

In [19]:
cur = conn.cursor()

In [20]:
engine = create_engine("postgresql+psycopg2://:@localhost/")

In [21]:
cur.execute("""
CREATE TABLE aisles (
    aisle_id INTEGER PRIMARY KEY,
    aisle VARCHAR
)
""")

In [22]:
cur.execute("""
CREATE TABLE departments (
    department_id INTEGER PRIMARY KEY,
    department VARCHAR(255)
)
""")

In [23]:
cur.execute("""
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    product_name VARCHAR(255),
    aisle_id INTEGER,
    department_id INTEGER,
    FOREIGN KEY (aisle_id) REFERENCES aisles (aisle_id),
    FOREIGN KEY (department_id) REFERENCES departments (department_id)
)
""")

In [24]:
cur.execute("""
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    order_number INTEGER,
    order_dow INTEGER,
    order_hour_of_day INTEGER,
    days_since_prior_order INTEGER )
""")

In [25]:
cur.execute("""
CREATE TABLE order_products (
    order_id INTEGER,
    product_id INTEGER,
    add_to_cart_order INTEGER,
    reordered INTEGER,
    PRIMARY KEY (order_id, product_id)
    )
""")

In [26]:
conn.commit()

In [27]:
order_df.drop('eval_set', inplace=True, axis=1)

In [28]:
aisles_df.to_sql('aisles', con=engine, if_exists='append', index=False)

134

In [29]:
departments_df.to_sql('departments', con=engine, if_exists='append', index=False)

21

In [30]:
products_df.to_sql('products', con=engine, if_exists='append', index=False)

688

In [31]:
order_df.to_sql('orders', con=engine, if_exists='append', index=False)

1000

In [32]:
order_products_df.to_sql('order_products', con=engine, if_exists='append', index=False)

1000

In [33]:
cur.close()

In [34]:
conn.close()